<a href="https://colab.research.google.com/github/annhienktuit/UIT-AI-Challenge2020/blob/master/YOLO_ObjectDetection_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Xây dựng bộ phát hiện đối tượng với thuật toán YOLOv3 được viết bằng thư viện Keras. Thư viện này bao gồm đầy đủ các module: phát hiện đối tượng với pre-trainded model, huấn luyện lại mô hình, phát hiện đối tượng ảnh trên ảnh/video/webcam.

Đầu tiên, ta sẽ khởi tạo Google Colab với sử dụng 1 GPU. Sau đó thực hiện kết nối với Google Drive của tài khoản cá nhân như sau:

In [1]:
!nvidia-smi

Fri Aug  7 12:40:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
lab_path = "/content/gdrive/My\ Drive/AdvancedComputerVision/keras-yolo3"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!ls

gdrive	sample_data


Sau đó, sử dụng git để clone thư mục project từ github của tác giả experiencor với đường dẫn như sau. Lưu ý rằng, ta nên đặt thực project kế bên file jupyter notebook này để tiện thao tác.

In [4]:
!git clone https://github.com/experiencor/keras-yolo3.git $lab_path

Cloning into '/content/gdrive/My Drive/AdvancedComputerVision/keras-yolo3'...
remote: Enumerating objects: 220, done.
remote: Total 220 (delta 0), reused 0 (delta 0), pack-reused 220
Receiving objects: 100% (220/220), 91.27 KiB | 1.79 MiB/s, done.
Resolving deltas: 100% (118/118), done.


Sau đó tiến hành cài đặt các thư viện cần thiết để có thể sử dụng được keras-yolo3. Đây là bước thực hiện quan trọng vì thông thường các thư viện sẽ sử dụng các thư viện bên thứ 3 với các phiên bản cố định. Do đó, để đảm bảo thư viện keras-yolo3 sử dụng trơn tru thì ta sẽ cài đặt các thư viện kèm theo trong file requirements.txt.

In [5]:
!pip install -r $lab_path/requirements.txt

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 2.4MB 9.2MB/s 
     |████████████████████████████████| 378kB 30.7MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 20.2MB 1.4MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 1.3MB 37.0MB/s 
     |████████████████████████████████| 276kB 30.0MB/s 
     |████████████████████████████████| 3.8MB 33.6MB/s 
     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 512kB 30.9MB/s 
     |████████████████████████████████| 327kB 30.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=944b3941bbc028b3d50d693ab45b99e76eeb415f95bc7ce67c4759bff1399113
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce774

## 1. Thử nghiệm với pretrained model của tác giả thuật toán YOLO.

File mô hình huấn luyện sẵn được đặt ở đường dẫn sau https://pjreddie.com/media/files/yolov3.weights
Lưu ý là file mô hình cần được đặt trong thư mục gốc của thư viện để tiện cho việc thực hiện.

In [4]:
!wget https://pjreddie.com/media/files/yolov3.weights -O $lab_path/yolov3.weights

--2020-08-07 12:41:14--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘/content/gdrive/My Drive/AdvancedComputerVision/keras-yolo3/yolov3.weights’

/content/gdrive/My  100%[===================>] 236.52M   397KB/s    in 9m 3s   

2020-08-07 12:50:19 (446 KB/s) - ‘/content/gdrive/My Drive/AdvancedComputerVision/keras-yolo3/yolov3.weights’ saved [248007048/248007048]



Sau đó, ta tiến hành sử dụng pretrained model để phát hiện đối tượng trong ảnh cho trước. Sử dụng chương trình yolo3_one_file_to_detect_them_all.py với file mô hình đã download ở file trước đó để phát hiện các đối tượng trong ảnh. Tập dữ liệu các đối tượng phát hiện được lấy từ tập PASCAL VOC 2012.

Ảnh kết quả phát hiện đối tượng trên pretrained model cho tập dữ liệu này được minh hoạ như ở hình sau:

![alt text](https://i.imgur.com/6nun5fl.jpg)

**Sửa np.nan thành sys.maxsize và import sys**

In [5]:
!python $lab_path/yolo3_one_file_to_detect_them_all.py -w $lab_path/yolov3.weights -i $lab_path/dog.jpg


Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-08-07 12:51:38.554490: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-07 12:51:38.609123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-07 12:51:38.609908: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2020-08-07 12:51:38.633046: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-08-07 12:51:38.855192: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-08-07 12:51:38.961658: I tensor

In [ ]:
from IPython.display import Image

Image('/content/gdrive/My Drive/AdvancedComputerVision/keras-yolo3_1/dog_detected.jpg')

#2. Huấn luyện dữ liệu với keras-yolo3

![alt text](https://i.imgur.com/45SRniL.jpg)

Bước tiếp theo, ta sẽ tiến hành huấn luyện trên tập dữ liệu mới là gấu mèo (raccoon). Tập dữ liệu này được tải từ https://github.com/experiencor/raccoon_dataset.git

Lưu ý rằng, ta sẽ chỉ sử dụng 2 thư mục của github này là images và annotations. Trong đó,

*   images: thư mục chứa các ảnh về gấu mèo
*   annotations: tương ứng với từng file ảnh của thư mục images, ta có một file .xml chứa thông tin về vị trí chính xác của đối tượng (gấu mèo) trong ảnh.

Các bạn nên xem kỹ định dạng của các file XML này để biết cấu trúc file gán nhãn phục vụ cho việc thực hiện đồ án sau này.



In [6]:
!git clone https://github.com/experiencor/raccoon_dataset.git $lab_path/raccoon_dataset

Cloning into '/content/gdrive/My Drive/AdvancedComputerVision/keras-yolo3/raccoon_dataset'...
remote: Enumerating objects: 646, done.
remote: Total 646 (delta 0), reused 0 (delta 0), pack-reused 646
Receiving objects: 100% (646/646), 48.00 MiB | 14.29 MiB/s, done.
Resolving deltas: 100% (412/412), done.
Checking out files: 100% (419/419), done.


In [ ]:
!ls

Một file khác cũng rất quan trọng của các mô hình huấn luyện học sâu nói chung và YOLO nói riêng là file cấu hình (config) các tham số khi huấn luyện.

Trong thư viện này, file config được đặt trong thư mục /zoo. Ví dụ như config_raccoon.json. Các tham số cơ bản của file config như:
- Đường dẫn đến thư mục train, validation (mặc định chia tỉ lệ train:validation = 8:2 nếu không có tham số gắn sẵn)
- GPU sử dụng để huấn luyện
- File pretrained cho tập image net backend.h5
- Learning rate
- Batch size

Sau khi đã cấu hình file config, ta sẽ tiến hành huấn luyện:

Để chương trình train có thể chạy được ta phải tải thêm file backend.h5 chứa pretrained model của backbone model và đặt ở thư mục lab_path:
https://drive.google.com/file/d/1f3Ylajkfg69iXc5159oJlYsq-9D_Px50/view?usp=sharing

**Chỉnh về 1 GPU, chỉnh path cho annotation và images(cả valid), chỉnh batch size về 8**

In [8]:
!python $lab_path/train.py -c $lab_path/zoo/config_raccoon.json

Using TensorFlow backend.
2020-08-07 13:06:06.194807: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-08-07 13:06:06.199906: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2020-08-07 13:06:06.200207: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1798bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-08-07 13:06:06.200301: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-08-07 13:06:06.202179: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-07 13:06:06.273091: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08

Sau khi quá trình huấn luyện kết thúc, hệ thống sẽ lưu mô hình raccoon.h5 trong thư mục của máy ảo colab. Ta có thể sửa lại mã nguồn của thư viện để các file mô hình và file trung gian đặt trong thư mục của thư viện cho gọn gàng.

Sau đó, ta sẽ tiến hành thử nghiệm mô hình đã huấn luyện trên ảnh mẫu lấy từ mạng internet hoặc thư mục ảnh validation/train.

In [13]:
!wget https://www.humanesociety.org/sites/default/files/styles/1240x698/public/2018/08/raccoon-440611.jpg

--2020-08-07 14:09:27--  https://www.humanesociety.org/sites/default/files/styles/1240x698/public/2018/08/raccoon-440611.jpg
Resolving www.humanesociety.org (www.humanesociety.org)... 23.185.0.1, 2620:12a:8001::1, 2620:12a:8000::1
Connecting to www.humanesociety.org (www.humanesociety.org)|23.185.0.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322378 (315K) [image/jpeg]
Saving to: ‘raccoon-440611.jpg’

raccoon-440611.jpg  100%[===================>] 314.82K  --.-KB/s    in 0.04s   

2020-08-07 14:09:27 (7.02 MB/s) - ‘raccoon-440611.jpg’ saved [322378/322378]



In [14]:
!python $lab_path/predict.py -c $lab_path/zoo/config_raccoon.json -i raccoon-440611.jpg

Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-08-07 14:09:47.629023: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-07 14:09:47.646754: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-07 14:09:47.647448: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2020-08-07 14:09:47.647761: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-08-07 14:09:47.649295: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-08-07 14:09:47.650471: I tensor

In [17]:
!cp -R "/content/raccoon.h5" "/content/gdrive/My Drive/Backup UIT"

#3. Train trên bộ dữ liệu giao thông

Như vậy, trong phần bài tập này chúng ta đã làm quen với thư viện keras-yolo3. Trong phần đồ án giữa kỳ, ta sẽ tiến hành trên dữ liệu video giao thông được thu tại TP. Hồ Chí Minh. Ví dụ, ảnh sau được cắt tại góc đường Nam Kỳ Khởi Nghĩa - Võ Thị Sáu.

![alt text](https://i.imgur.com/NQBeNpG.jpg)

Mục tiêu của đồ án này là sử dụng thuật toán YOLOv3 để tiến hành ước lượng lưu lượng giao thông theo các hướng đi.

Đầu tiên, ta sẽ thử sử dụng pretrained model lên dữ liệu video thực tế.

Sau đó tiến hành xây dựng bộ dữ liệu gán nhãn cho ngữ cảnh camera giao thông.

Huấn luyện lại mô hình với dữ liệu gán nhãn mới.

Phát hiện và đếm các đối tượng xe máy, xe hơi trong video giao thông.

Video test: https://drive.google.com/file/d/1u__2tl79nSevF_pnvwsVd79Hu1aWLJQE/view?usp=sharing

Tập ảnh dùng để train: https://drive.google.com/file/d/1-ZT6V2OrIWIWtLFFlizQBVvYxL__QFL7/view?usp=sharing

